# MNIST Dataset handwriting recognition with DNN

In [16]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Flatten, Dropout
import numpy as np
from keras.utils import np_utils

## Load data

In [4]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)

(60000, 28, 28)
(60000,)


## Define NN structure

In [5]:
model = Sequential()
model.add(Dense(input_dim=28*28, output_dim=512, activation='relu'))
model.add(Dense(output_dim=512, activation='relu'))
model.add(Dense(output_dim=10, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
_________________________________________________________________


/home/sean/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=784, activation="relu", units=512)`
  
/home/sean/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=512)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/sean/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=10)`
  after removing the cwd from sys.path.


## Restructure Training Data and Target structure and fit to model

In [6]:
x_train_flatten = x_train.reshape(-1,28*28)
x_train_flatten = x_train_flatten.astype('float32')
x_train_flatten /= 255
y_train_vectorized = np_utils.to_categorical(y_train,num_classes=10)
model.fit(x_train_flatten, y_train_vectorized, batch_size= 20, epochs= 5 )

Epoch 1/5
60000/60000 [==============================] - 41s 681us/step - loss: 0.1823 - acc: 0.9450
Epoch 2/5
60000/60000 [==============================] - 39s 642us/step - loss: 0.0866 - acc: 0.9732
Epoch 3/5
60000/60000 [==============================] - 42s 699us/step - loss: 0.0591 - acc: 0.9817
Epoch 4/5
60000/60000 [==============================] - 44s 736us/step - loss: 0.0480 - acc: 0.9849
Epoch 5/5
60000/60000 [==============================] - 45s 754us/step - loss: 0.0403 - acc: 0.9877


## Evaluate accuracy

In [7]:
x_test_flatten = x_test.reshape(-1,28*28)
x_test_flatten = x_test_flatten.astype('float32')
x_test_flatten /= 255
y_test_vectorized = np_utils.to_categorical(y_test,num_classes=10)
result = model.predict(x_test_flatten)
result = np.argmax(result,axis=1)
label = np.argmax(y_test_vectorized,axis=1)
accu = np.sum(result==label)/len(result)*100
print(accu)

97.11999999999999


## Now use CNN structure

In [25]:
model2 = Sequential()
model2.add(Conv2D(32,(3,3),input_shape =(28,28,1),activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Conv2D(32,(3,3),activation='relu'))
model2.add(MaxPooling2D(pool_size=(2,2)))
model2.add(Flatten())
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(128, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))

model2.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 800)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               205056    
_________________________________________________________________
dropout_5 (Dropout)          (None, 256)               0         
__________

## fit to model and train. features do not need to reshape

In [27]:
x_train_norm = x_train.reshape(-1,28,28,1)/255
#print(x_test_norm[0])
model2.fit(x_train_norm,y_train_vectorized, batch_size=20, epochs=5 )

Epoch 1/5
60000/60000 [==============================] - 46s 761us/step - loss: 0.1569 - acc: 0.9499
Epoch 2/5
60000/60000 [==============================] - 45s 754us/step - loss: 0.0556 - acc: 0.9836
Epoch 3/5
60000/60000 [==============================] - 46s 768us/step - loss: 0.0392 - acc: 0.9883
Epoch 4/5
60000/60000 [==============================] - 44s 730us/step - loss: 0.0318 - acc: 0.9902
Epoch 5/5
60000/60000 [==============================] - 46s 765us/step - loss: 0.0264 - acc: 0.9922


In [28]:
x_test_norm = x_test.reshape(-1,28,28,1)/255
result2 = model2.predict(x_test_norm)
result2 = np.argmax(result2,axis=1)
accu2 = np.sum(result2==label)/len(result2)*100
print(accu2)

99.09


Conclution: CNN with less neuron 248k vs 670k performs better than DNN (99% vs 97.1)